In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from torchvision.transforms import Compose, ToTensor, Normalize,Resize ,CenterCrop ,Grayscale
from torch.utils.data import DataLoader

def train_val_dataset(dataset, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets
m = [0.485, 0.456, 0.406]
s = [0.229, 0.224, 0.225]
dataset = ImageFolder(r"/content/gdrive/MyDrive/Datasets/mnistTask", transform=Compose([Resize(256),
        CenterCrop(224),ToTensor(),Normalize(                      
                         mean = m,
                         std = s
                     )  ]))
print(len(dataset))
datasets = train_val_dataset(dataset)
print(len(datasets['train']))
print(len(datasets['val']))

In [ ]:
import cv2                
import numpy as np
import matplotlib.pyplot as plt                        
%matplotlib inline    
dataloaders = {x:DataLoader(datasets[x],16, shuffle=True, num_workers=0) for x in ['train','val']}
x,y = next(iter(dataloaders['train']))
print(x.shape, y.shape)

import torchvision
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) 

inputs, classes = next(iter(dataloaders['train']))
out = torchvision.utils.make_grid(inputs)
imshow(out , title = classes+1)

In [ ]:
use_cuda = torch.cuda.is_available()
print(use_cuda)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,32,3,padding =1)
        self.conv1_bn = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32,64,3,padding =1)
        self.conv2_bn = nn.BatchNorm2d(128)
        #self.conv3 = nn.Conv2d(128,256,3,padding =1)
        #self.conv3_bn = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(56 * 56 * 64, 512) #28*28*256
        self.fc2 = nn.Linear(512, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1_bn(self.conv1(x))))
        x = self.pool(F.relu(self.conv2_bn(self.conv2(x))))
        #x = self.pool(F.relu(self.conv3_bn(self.conv3(x))))
        x= x.view(x.size(0), -1)
        x = self.dropout(x)
        x= self.fc1(x)
        x= self.fc2(x)
        return x


model_scratch_mnist = Net()
if use_cuda:
    model_scratch_mnist.cuda()

In [ ]:
import torch.optim as optim

criterion_scratch = nn.CrossEntropyLoss()
optimizer_scratch = optim.Adam(model_scratch_mnist.parameters() , lr = 1e-3)

In [ ]:
import time
n_epochs_stop = 5
epochs_no_improve = 0

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    
    since = time.time()
    valid_loss_min = np.Inf 
    running_corrects = 0
    
    for epoch in range(1 , n_epochs+1):
        train_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            out = model(data)
            loss = criterion(out,target)
            loss.backward()
            optimizer.step()
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['val']):
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            outputs = model(data)
            loss = criterion(outputs, target)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
          
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds  == target.data)

        epoch_acc = running_corrects.double() / len(datasets['val'])
    
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tEpoc Accuracy: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss,
            epoch_acc
            ))
        
        if valid_loss < valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}). Saving the model'.format(valid_loss_min, valid_loss))
            torch.save(model.state_dict(), save_path)
            epochs_no_improve = 0
            valid_loss_min = valid_loss 
        else:
            epochs_no_improve += 1
            if epochs_no_improve == n_epochs_stop:
              print('Early stopping!')
              time_elapsed = time.time() - since
              return model
              print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))    
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
            
    return model


In [ ]:
model_scratch_mnist = train(15, dataloaders, model_scratch_mnist, optimizer_scratch, 
                      criterion_scratch, use_cuda, r'/content/gdrive/MyDrive/Saved models/model_scratch_mnistTask.pt')

In [ ]:

model_scratch_mnist.load_state_dict(torch.load('/content/gdrive/MyDrive/Saved models/model_scratch_mnistTask.pt'))

In [ ]:
import idx2numpy
import numpy as np
import torch
from torch.utils.data import Dataset, TensorDataset
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
%matplotlib inline

file_image = '/content/gdrive/MyDrive/Datasets/train-images.idx3-ubyte'
file_label = '/content/gdrive/MyDrive/Datasets/t10k-labels.idx1-ubyte'
X_test = idx2numpy.convert_from_file(file_image)
X_test = torch.stack([torch.from_numpy(np.array(i)) for i in X_test])
X_test = X_test.reshape((-1, 1, 28, 28))
X_test = torch.tensor(X_test)
Y_test = idx2numpy.convert_from_file(file_label)
Y_test = Y_test.reshape(Y_test.shape[0], 1)
Y_test = torch.tensor(Y_test)

class CustomTensorDataset(Dataset):
    
    def __init__(self, tensors, transform=None):
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        x = self.tensors[0][index]

        if self.transform:
            x = self.transform(x)

        y = self.tensors[1][index]

        return x, y

    def __len__(self):
        return self.tensors[0].size(0)


def imshow(img, title=''):
    plt.figure(figsize=(10, 10))
    plt.title(title)
    plt.imshow(np.transpose( img.numpy(), (1, 2, 0)), cmap='gray')
    plt.show()


test_dataset_normal = CustomTensorDataset(tensors=(X_test,Y_test), transform=Compose([Resize(256), CenterCrop(224),ToTensor()])))
test_loader = torch.utils.data.DataLoader(test_dataset_normal, batch_size=16)

#outputs = model_scratch_mnist(X_test) #fix this

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model_scratch_mnist(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(10):#4
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

#with pre trained model from 1

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,32,3,padding =1)
        self.conv1_bn = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32,128,3,padding =1)
        self.conv2_bn = nn.BatchNorm2d(128)
        #self.conv3 = nn.Conv2d(128,256,3,padding =1)
        #self.conv3_bn = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(56 * 56 * 128, 1000) #28*28*256
        self.fc2 = nn.Linear(1000, 62)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1_bn(self.conv1(x))))
        x = self.pool(F.relu(self.conv2_bn(self.conv2(x))))
        #x = self.pool(F.relu(self.conv3_bn(self.conv3(x))))
        x= x.view(x.size(0), -1)
        x = self.dropout(x)
        x= self.fc1(x)
        x= self.fc2(x)
        return x


model_scratch1_imported = Net()
if use_cuda:
    model_scratch1_imported.cuda()

In [ ]:
for param in model_scratch1_imported.parameters():
  param.require_grad = False
fc = nn.Sequential(
    nn.Linear((56 * 56 * 128, 800),
    nn.ReLU(),
    nn.Dropout(0.4),
    
    nn.Linear(800,10),
    nn.LogSoftmax(dim=1)
    
)
model_scratch1_imported.classifier = fc
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model_scratch1_imported.classifier.parameters(), lr=0.003)
if use_cuda:
    model_scratch1_imported.cuda()

In [ ]:
import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from torchvision.transforms import Compose, ToTensor, Normalize,Resize ,CenterCrop ,Grayscale
from torch.utils.data import DataLoader

def train_val_dataset(dataset, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets
m = [0.485, 0.456, 0.406]
s = [0.229, 0.224, 0.225]
dataset = ImageFolder(r"/content/gdrive/MyDrive/Datasets/mnistTask", transform=Compose([Resize(256),
        CenterCrop(224),ToTensor(),Normalize(                      
                         mean = m,
                         std = s
                     )  ]))
print(len(dataset))
datasets = train_val_dataset(dataset)
print(len(datasets['train']))
print(len(datasets['val']))

In [ ]:
import cv2                
import numpy as np
import matplotlib.pyplot as plt                        
%matplotlib inline    
dataloaders = {x:DataLoader(datasets[x],16, shuffle=True, num_workers=0) for x in ['train','val']}
x,y = next(iter(dataloaders['train']))
print(x.shape, y.shape)

import torchvision
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) 

inputs, classes = next(iter(dataloaders['train']))
out = torchvision.utils.make_grid(inputs)
imshow(out , title = classes+1)

In [ ]:
import torch.optim as optim

criterion_scratch = nn.CrossEntropyLoss()
optimizer_scratch = optim.Adam(model_scratch.classifier.parameters() , lr = 1e-3)

In [ ]:
model_transfer =  train(20, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_scratch1_imported_for_c.pt')